In [2]:
#https://scipy-cookbook.readthedocs.io/items/Matplotlib_Gridding_irregularly_spaced_data.html#

import shapely.geometry
import pyproj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics as st
import time as time
from datetime import datetime
from datetime import timedelta
from median_absolute_deviation_calculator import median_absolute_deviation_calculator

# Set up projections
p_ll = pyproj.Proj(init='epsg:4326')#3310 #3488
p_mt = pyproj.Proj(init='epsg:3857') # metric; same as EPSG:900913
# Create corners of rectangle to be transformed to a grid
#sw = shapely.geometry.Point((-127, 30.0))
#ne = shapely.geometry.Point((-116, 50.0))
sw = shapely.geometry.Point((-126, 32.0))
ne = shapely.geometry.Point((-117, 47.0))
binsize = 5000 # 5 km grid step size

#plt_east = -118
#plt_west = -125
#plt_north = 42
#plt_south = 32

# Project corners to target projection
transformed_sw = pyproj.transform(p_ll, p_mt, sw.x, sw.y) # Transform NW point to 3857
transformed_ne = pyproj.transform(p_ll, p_mt, ne.x, ne.y) # .. same for SE

#print(transformed_sw)
#print(transformed_ne)

# get extrema values.
lonmin, lonmax = transformed_sw[0], transformed_ne[0]
latmin, latmax = transformed_sw[1], transformed_ne[1]

print('Making Coordinate Arrays & Grids')
# make coordinate arrays.
xi = np.arange(lonmin, lonmax+binsize, binsize)
yi = np.arange(latmin, latmax+binsize, binsize)
xi, yi = np.meshgrid(xi,yi)


# make the grid to hold gridded data
#Total Krill Biomass
bio_mean = np.zeros(xi.shape, dtype=xi.dtype)
bio_med=np.copy(bio_mean)
bio_std=np.copy(bio_mean)
bio_mad=np.copy(bio_mean)

#Total Day Only Krill Biomass 
bioday_mean=np.copy(bio_mean)
bioday_med=np.copy(bio_mean)
bioday_std=np.copy(bio_mean)
bioday_mad=np.copy(bio_mean)

#Total Log Krill Biomass
logbio_mean=np.copy(bio_mean)
logbio_med=np.copy(bio_mean)
logbio_std=np.copy(bio_mean)
logbio_mad=np.copy(bio_mean)

#Day Only Krill Biomass
logbioday_mean=np.copy(bio_mean)
logbioday_med=np.copy(bio_mean)
logbioday_std=np.copy(bio_mean)
logbioday_mad=np.copy(bio_mean)

#E.pacifica Day Only
ep_bioday_mean=np.copy(bio_mean)
ep_bioday_med=np.copy(bio_mean)
ep_bioday_std=np.copy(bio_mean)
ep_bioday_mad=np.copy(bio_mean)
ep_logbioday_mean=np.copy(bio_mean)
ep_logbioday_med=np.copy(bio_mean)
ep_logbioday_std=np.copy(bio_mean)
ep_logbioday_mad=np.copy(bio_mean)

#T.spinifera Day Only
ts_bioday_mean=np.copy(bio_mean)
ts_bioday_med=np.copy(bio_mean)
ts_bioday_std=np.copy(bio_mean)
ts_bioday_mad=np.copy(bio_mean)
ts_logbioday_mean=np.copy(bio_mean)
ts_logbioday_med=np.copy(bio_mean)
ts_logbioday_std=np.copy(bio_mean)
ts_logbioday_mad=np.copy(bio_mean)

#N.difficilis Day Only
nd_bioday_mean=np.copy(bio_mean)
nd_bioday_med=np.copy(bio_mean)
nd_bioday_std=np.copy(bio_mean)
nd_bioday_mad=np.copy(bio_mean)
nd_logbioday_mean=np.copy(bio_mean)
nd_logbioday_med=np.copy(bio_mean)
nd_logbioday_std=np.copy(bio_mean)
nd_logbioday_mad=np.copy(bio_mean)

#Number of samples
num_samples = np.copy(bio_mean)
num_samples_day = np.copy(bio_mean)

#Number of Years of Data
num_years= np.copy(bio_mean)
num_years_day= np.copy(bio_mean)

# create list in same shape as grid to store indices
wherebin = np.copy(bio_mean)
wherebin = wherebin.tolist()

nrow, ncol = bio_mean.shape


print('Opening Data')
base='/Users/jeffdorman/Documents/python/krill_biomass/'
fin = base + 'output_data/biomass/allyears_bio_depth_integrated.csv'
df1=pd.read_csv(fin)
df1=df1.replace([-9999.0, 9999.0, -999.0, -9998], np.nan)


#time_str=df1['Time_M']
#dtime=list()
#for i in range(len(df1['Time_M'])):
#    tmptime=df1.loc[i, 'Time_M']
#    tmpdate=str(df1.loc[i, 'Date_M'])
#    combo=tmpdate+tmptime
#    dtime.append(datetime.strptime(combo, "%Y%m%d %H:%M:%S.%f"))

df1['d_time_local']=pd.to_datetime(df1['d_time_local'], errors='coerce')



bio=df1['krill_biomass_fromABC']
bioday=df1['krill_biomass_fromABC_dayonly']
logbio=df1['log10_krill_biomass_fromABC']
logbioday=df1['log10_krill_biomass_fromABC_dayonly']

ep_bioday=df1['epac_biomass_fromABC_dayonly']
ts_bioday=df1['tspin_biomass_fromABC_dayonly']
nd_bioday=df1['ndiff_biomass_fromABC_dayonly']

ep_logbioday=df1['log10_epac_biomass_fromABC_dayonly']
ts_logbioday=df1['log10_tspin_biomass_fromABC_dayonly']
nd_logbioday=df1['log10_ndiff_biomass_fromABC_dayonly']

year=df1['d_time_local'].dt.year


##Not Needed as this is now saved int he file
#print('Transforming Lat/Lon')
#transxy=pyproj.transform(p_ll, p_mt, lon, lat)
#x=transxy[0]
#y=transxy[1]
x=df1['lon_meters']
y=df1['lat_meters']

#print(np.size(x))
#print(np.size(y))
#print(np.size(z))

print('Calculating')
#row=18
#col=164
for row in range(nrow):#nrow
    for col in range(ncol):#ncol
        xc = xi[row, col]    # x coordinate.
        yc = yi[row, col]    # y coordinate.

        # find the position that xc and yc correspond to.
        posx = np.abs(x - xc)
        posy = np.abs(y - yc)
        ibin = np.logical_and(posx < binsize/2., posy < binsize/2.)
        ind  = np.where(ibin == True)[0]

        # fill the bin.
        bin_bio = bio[ibin]
        bin_bioday=bioday[ibin]
        bin_logbio = logbio[ibin]
        bin_logbioday = logbioday[ibin]
        
        ep_bin_bioday = ep_bioday[ibin]
        ep_bin_logbioday = ep_logbioday[ibin]
        ts_bin_bioday = ts_bioday[ibin]
        ts_bin_logbioday = ts_logbioday[ibin]
        nd_bin_bioday = nd_bioday[ibin]
        nd_bin_logbioday = nd_logbioday[ibin]
        
        bin_year=year[ibin]
        bin_year_day=year[ibin]
        #print(bin_year)
        #print(bin_year_day)
        #print(bin_bio)

        wherebin[row][col] = ind

        num_samples[row, col] = sum(~np.isnan(bin_bio))
        num_samples_day[row, col] = sum(~np.isnan(bin_bioday))

        #bio_loar=np.isnan(bin_bio)
        #bioday_loar=np.isnan(bin_bioday)

        if bin_bio.size != 0:
            if ~np.isnan(bin_bio).all():
                #print('row = ' + str(row))
                #print('col = ' + str(col))
                bio_mean[row, col] = np.nanmean(bin_bio)
                bio_med[row, col] = np.nanmedian(bin_bio)
                bio_std[row, col] = np.nanstd(bin_bio)
                bio_mad[row, col] = median_absolute_deviation_calculator(bio_med[row,col],bin_bio)

                logbio_mean[row, col] = np.nanmean(bin_logbio)
                logbio_med[row, col] = np.nanmedian(bin_logbio)
                logbio_std[row, col] = np.nanstd(bin_logbio)
                logbio_mad[row, col] = median_absolute_deviation_calculator(logbio_med[row,col],bin_logbio)

                #Remove the np.nans from the Year
                if np.any(np.isnan(bin_bio)): #if you have any nan's in bin_bio
                    bio_loar=np.isnan(bin_bio)  #find out where they are
                    bin_year[bio_loar]=np.nan   #Turn the years into np.nan's
                    num_years[row,col]=len(np.unique(bin_year[~np.isnan(bin_year)]))
                    #num_years[row,col]=len(np.unique(bin_year))-1 #subtract 1 becuase unique counts the np.nan as one instance
                else:
                    num_years[row,col]=len(np.unique(bin_year))
            else:
                bio_mean[row, col] = np.nan
                bio_med[row, col] = np.nan
                bio_std[row, col] = np.nan
                bio_mad[row, col] = np.nan

                logbio_mean[row, col] = np.nan
                logbio_med[row, col] = np.nan
                logbio_std[row, col] = np.nan 
                logbio_mad[row, col] = np.nan
            
                num_years[row,col]=np.nan
                
        else:  # fill empty bins with nans
            bio_mean[row, col] = np.nan
            bio_med[row, col] = np.nan
            bio_std[row, col] = np.nan
            bio_mad[row, col] = np.nan

            logbio_mean[row, col] = np.nan
            logbio_med[row, col] = np.nan
            logbio_std[row, col] = np.nan 
            logbio_mad[row, col] = np.nan
            
            num_years[row,col]=np.nan

            
        if bin_bioday.size != 0:
            if ~np.isnan(bin_bioday).all():

                bioday_mean[row, col] = np.nanmean(bin_bioday)
                bioday_med[row, col] = np.nanmedian(bin_bioday)
                bioday_std[row, col] = np.nanstd(bin_bioday)
                bioday_mad[row, col] = median_absolute_deviation_calculator(bioday_med[row,col],bin_bioday)

                logbioday_mean[row, col] = np.nanmean(bin_logbioday)
                logbioday_med[row, col] = np.nanmedian(bin_logbioday)
                logbioday_std[row, col] = np.nanstd(bin_logbioday)
                logbioday_mad[row, col] = median_absolute_deviation_calculator(logbioday_med[row,col],bin_logbioday)
                
                ep_bioday_mean[row, col] = np.nanmean(ep_bin_bioday)
                ep_bioday_med[row, col] = np.nanmedian(ep_bin_bioday)
                ep_bioday_std[row, col] = np.nanstd(ep_bin_bioday)
                ep_bioday_mad[row, col] = median_absolute_deviation_calculator(ep_bioday_med[row,col],ep_bin_bioday)
                ep_logbioday_mean[row, col] = np.nanmean(ep_bin_logbioday)
                ep_logbioday_med[row, col] = np.nanmedian(ep_bin_logbioday)
                ep_logbioday_std[row, col] = np.nanstd(ep_bin_logbioday)
                ep_logbioday_mad[row, col] = median_absolute_deviation_calculator(ep_logbioday_med[row,col],ep_bin_logbioday)

                ts_bioday_mean[row, col] = np.nanmean(ts_bin_bioday)
                ts_bioday_med[row, col] = np.nanmedian(ts_bin_bioday)
                ts_bioday_std[row, col] = np.nanstd(ts_bin_bioday)
                ts_bioday_mad[row, col] = median_absolute_deviation_calculator(ts_bioday_med[row,col],ts_bin_bioday)
                ts_logbioday_mean[row, col] = np.nanmean(ts_bin_logbioday)
                ts_logbioday_med[row, col] = np.nanmedian(ts_bin_logbioday)
                ts_logbioday_std[row, col] = np.nanstd(ts_bin_logbioday)
                ts_logbioday_mad[row, col] = median_absolute_deviation_calculator(ts_logbioday_med[row,col],ts_bin_logbioday)
                
                nd_bioday_mean[row, col] = np.nanmean(nd_bin_bioday)
                nd_bioday_med[row, col] = np.nanmedian(nd_bin_bioday)
                nd_bioday_std[row, col] = np.nanstd(nd_bin_bioday)
                nd_bioday_mad[row, col] = median_absolute_deviation_calculator(nd_bioday_med[row,col],nd_bin_bioday)
                nd_logbioday_mean[row, col] = np.nanmean(nd_bin_logbioday)
                nd_logbioday_med[row, col] = np.nanmedian(nd_bin_logbioday)
                nd_logbioday_std[row, col] = np.nanstd(nd_bin_logbioday)
                nd_logbioday_mad[row, col] = median_absolute_deviation_calculator(nd_logbioday_med[row,col],nd_bin_logbioday)
                
                if np.any(np.isnan(bin_bioday)): #if you have any nan's in bin_bio
                    bioday_loar=np.isnan(bin_bioday)  #find out where they are
                    bin_year_day[bioday_loar]=np.nan   #Turn the years into np.nan's
                    num_years_day[row,col]=len(np.unique(bin_year_day[~np.isnan(bin_year_day)]))
                    #num_years_day[row,col]=len(np.unique(bin_year_day))-1 #subtract 1 becuase unique counts the np.nan as one instance
                else:
                    num_years_day[row,col]=len(np.unique(bin_year_day))
            else:
                bioday_mean[row, col] = np.nan
                bioday_med[row, col] = np.nan
                bioday_std[row, col] = np.nan
                bioday_mad[row, col] = np.nan

                logbioday_mean[row, col] = np.nan
                logbioday_med[row, col] = np.nan
                logbioday_std[row, col] = np.nan 
                logbioday_mad[row, col] = np.nan
                
                ep_bioday_mean[row, col] = np.nan
                ep_bioday_med[row, col] = np.nan
                ep_bioday_std[row, col] = np.nan
                ep_bioday_mad[row, col] = np.nan
                ep_logbioday_mean[row, col] = np.nan
                ep_logbioday_med[row, col] = np.nan
                ep_logbioday_std[row, col] = np.nan 
                ep_logbioday_mad[row, col] = np.nan
                
                ts_bioday_mean[row, col] = np.nan
                ts_bioday_med[row, col] = np.nan
                ts_bioday_std[row, col] = np.nan
                ts_bioday_mad[row, col] = np.nan
                ts_logbioday_mean[row, col] = np.nan
                ts_logbioday_med[row, col] = np.nan
                ts_logbioday_std[row, col] = np.nan 
                ts_logbioday_mad[row, col] = np.nan
                
                nd_bioday_mean[row, col] = np.nan
                nd_bioday_med[row, col] = np.nan
                nd_bioday_std[row, col] = np.nan
                nd_bioday_mad[row, col] = np.nan
                nd_logbioday_mean[row, col] = np.nan
                nd_logbioday_med[row, col] = np.nan
                nd_logbioday_std[row, col] = np.nan 
                nd_logbioday_mad[row, col] = np.nan
            
                num_years_day[row,col]=np.nan
        
        else: # fill empty bins with nans
            bioday_mean[row, col] = np.nan
            bioday_med[row, col] = np.nan
            bioday_std[row, col] = np.nan
            bioday_mad[row, col] = np.nan

            logbioday_mean[row, col] = np.nan
            logbioday_med[row, col] = np.nan
            logbioday_std[row, col] = np.nan 
            logbioday_mad[row, col] = np.nan
            
            ep_bioday_mean[row, col] = np.nan
            ep_bioday_med[row, col] = np.nan
            ep_bioday_std[row, col] = np.nan
            ep_bioday_mad[row, col] = np.nan
            ep_logbioday_mean[row, col] = np.nan
            ep_logbioday_med[row, col] = np.nan
            ep_logbioday_std[row, col] = np.nan 
            ep_logbioday_mad[row, col] = np.nan

            ts_bioday_mean[row, col] = np.nan
            ts_bioday_med[row, col] = np.nan
            ts_bioday_std[row, col] = np.nan
            ts_bioday_mad[row, col] = np.nan
            ts_logbioday_mean[row, col] = np.nan
            ts_logbioday_med[row, col] = np.nan
            ts_logbioday_std[row, col] = np.nan 
            ts_logbioday_mad[row, col] = np.nan

            nd_bioday_mean[row, col] = np.nan
            nd_bioday_med[row, col] = np.nan
            nd_bioday_std[row, col] = np.nan
            nd_bioday_mad[row, col] = np.nan
            nd_logbioday_mean[row, col] = np.nan
            nd_logbioday_med[row, col] = np.nan
            nd_logbioday_std[row, col] = np.nan 
            nd_logbioday_mad[row, col] = np.nan
            
            num_years_day[row,col]=np.nan 
 
        if num_years_day[row,col]>7:
            print('num_years_day')
            print('row = ' + str(row))
            print('col = ' + str(col)) 
        if num_years[row,col]>7:
            print('num_years')
            print('row = ' + str(row))
            print('col = ' + str(col)) 


print('back_transforming the grid')                
bt=pyproj.transform(p_mt, p_ll, xi, yi)
print('finished')
            
            


/Users/jeffdorman/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/jeffdorman/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/Users/jeffdorman/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initializatio

Making Coordinate Arrays & Grids
Opening Data
Calculating
back_transforming the grid
finished


/Users/jeffdorman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:397: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


In [3]:
np.save('gridded_data/bio_mean.npy', bio_mean)
np.save('gridded_data/bio_med.npy', bio_med)
np.save('gridded_data/bio_std.npy', bio_std)
np.save('gridded_data/bio_mad.npy', bio_mad)

np.save('gridded_data/logbio_mean.npy', logbio_mean)
np.save('gridded_data/logbio_med.npy', logbio_med)
np.save('gridded_data/logbio_std.npy', logbio_std)
np.save('gridded_data/logbio_mad.npy', logbio_mad)

np.save('gridded_data/bioday_mean.npy', bioday_mean)
np.save('gridded_data/bioday_med.npy', bioday_med)
np.save('gridded_data/bioday_std.npy', bioday_std)
np.save('gridded_data/bioday_mad.npy', bioday_mad)

np.save('gridded_data/logbioday_mean.npy', logbioday_mean)
np.save('gridded_data/logbioday_med.npy', logbioday_med)
np.save('gridded_data/logbioday_std.npy', logbioday_std)
np.save('gridded_data/logbioday_mad.npy', logbioday_mad)

np.save('gridded_data/ep_bioday_mean.npy', ep_bioday_mean)
np.save('gridded_data/ep_bioday_med.npy', ep_bioday_med)
np.save('gridded_data/ep_bioday_std.npy', ep_bioday_std)
np.save('gridded_data/ep_bioday_mad.npy', ep_bioday_mad)
np.save('gridded_data/ep_logbioday_mean.npy', ep_logbioday_mean)
np.save('gridded_data/ep_logbioday_med.npy', ep_logbioday_med)
np.save('gridded_data/ep_logbioday_std.npy', ep_logbioday_std)
np.save('gridded_data/ep_logbioday_mad.npy', ep_logbioday_mad)

np.save('gridded_data/ts_bioday_mean.npy', ts_bioday_mean)
np.save('gridded_data/ts_bioday_med.npy', ts_bioday_med)
np.save('gridded_data/ts_bioday_std.npy', ts_bioday_std)
np.save('gridded_data/ts_bioday_mad.npy', ts_bioday_mad)
np.save('gridded_data/ts_logbioday_mean.npy', ts_logbioday_mean)
np.save('gridded_data/ts_logbioday_med.npy', ts_logbioday_med)
np.save('gridded_data/ts_logbioday_std.npy', ts_logbioday_std)
np.save('gridded_data/ts_logbioday_mad.npy', ts_logbioday_mad)

np.save('gridded_data/nd_bioday_mean.npy', nd_bioday_mean)
np.save('gridded_data/nd_bioday_med.npy', nd_bioday_med)
np.save('gridded_data/nd_bioday_std.npy', nd_bioday_std)
np.save('gridded_data/nd_bioday_mad.npy', nd_bioday_mad)
np.save('gridded_data/nd_logbioday_mean.npy', nd_logbioday_mean)
np.save('gridded_data/nd_logbioday_med.npy', nd_logbioday_med)
np.save('gridded_data/nd_logbioday_std.npy', nd_logbioday_std)
np.save('gridded_data/nd_logbioday_mad.npy', nd_logbioday_mad)

np.save('gridded_data/num_samples.npy', num_samples)
np.save('gridded_data/num_samples_day.npy', num_samples_day)

np.save('gridded_data/num_years.npy', num_years)
np.save('gridded_data/num_years_day.npy', num_years_day)

np.save('gridded_data/lat.npy', bt[1])
np.save('gridded_data/lon.npy', bt[0])

In [ ]:
import numpy as np
jeff=[np.nan, np.nan, np.nan]
tess=np.nanmean(jeff)
print(tess)